## Forecasting using Markov Chain

Models:
1. Get the sales array from the last year for the category/product to be forecasted
2. Classify the sales values into discrete states (e.g 100 states)
3. Calculate the transition matrix based on last year sales array
4. Make the forecast

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [329]:
### intialise the parameters
low = 0
high = 200
week_size = 52
n_states = 20

#### Step 1

In [374]:
np.random.seed(1)
ly_act_week_sales = np.random.randint(low=low, high=high, size=week_size)

In [331]:
ly_act_week_sales

array([ 37, 140,  72, 137, 133,  79, 192, 144, 129,  71, 134,  25, 178,
        20, 101, 146, 139, 156, 157, 142,  50,  68,  96,  86, 141, 137,
         7,  63,  61,  22,  57,   1, 128,  60,   8, 141, 115, 175, 121,
        30,  71, 131, 198, 149,  49,  57,   3, 196,  24,  43,  76,  26])

#### Step 2

In [421]:
def sales_state_class(series, low, high, n_bins):
    arr_bins = np.zeros(n_bins+1)
    
    interval = (high - low) / n_bins
    arr_bins[0] = low
    for i in range(1, n_bins+1):
        arr_bins[i] = arr_bins[i-1] + interval
    #print(arr_bins)
    
    sales_state_map = np.zeros(len(arr_bins)-1)
    for (i,j) in zip(range(0, len(arr_bins)), range(1, len(arr_bins))):
        #print(i, j)
        sales_state_map[i] = (arr_bins[i] + arr_bins[j])/2 
    
    # bucket starts from 1, -1 to make it start from 0
    return np.digitize(series, arr_bins, right=True)-1, sales_state_map

In [423]:
sales_state_class(ly_act_week_sales, low, high, n_states)[1]

array([  5.,  15.,  25.,  35.,  45.,  55.,  65.,  75.,  85.,  95., 105.,
       115., 125., 135., 145., 155., 165., 175., 185., 195.])

In [468]:
ly_act_week_sales_state, ly_act_week_sales_fcast = sales_state_class(ly_act_week_sales, low, high, n_states)

#### Step 3

In [360]:
def transition_matrix(transitions, n_states):
    #n = np.unique(transitions).shape[0] #number of states

    M = np.zeros((n_states, n_states))

    for (i,j) in zip(transitions[:-1], transitions[1:]):
        #print(i,j)
        M[i, j] += 1

    #now convert to probabilities:
    for row in M:
        s = np.sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

In [361]:
ly_act_week_sales_state

array([ 3, 13,  7, 13, 13,  7, 19, 14, 12,  7, 13,  2, 17,  1, 10, 14, 13,
       15, 15, 14,  4,  6,  9,  8, 14, 13,  0,  6,  6,  2,  5,  0, 12,  5,
        0, 14, 11, 17, 12,  2,  7, 13, 19, 14,  4,  5,  0, 19,  2,  4,  7,
        2])

In [362]:
ly_act_week_sales_state[:-1]

array([ 3, 13,  7, 13, 13,  7, 19, 14, 12,  7, 13,  2, 17,  1, 10, 14, 13,
       15, 15, 14,  4,  6,  9,  8, 14, 13,  0,  6,  6,  2,  5,  0, 12,  5,
        0, 14, 11, 17, 12,  2,  7, 13, 19, 14,  4,  5,  0, 19,  2,  4,  7])

In [363]:
ly_act_week_sales_state[1:]

array([13,  7, 13, 13,  7, 19, 14, 12,  7, 13,  2, 17,  1, 10, 14, 13, 15,
       15, 14,  4,  6,  9,  8, 14, 13,  0,  6,  6,  2,  5,  0, 12,  5,  0,
       14, 11, 17, 12,  2,  7, 13, 19, 14,  4,  5,  0, 19,  2,  4,  7,  2])

In [364]:
M = transition_matrix(ly_act_week_sales_state, n_states)

#### Step 4

In [369]:
def sales_forecast(init_state, transition_matrix, step, n_states):
    fcast_week_state = np.zeros(step+1)
    fcast_week_state[0] = np.int(init_state)
    
    for i in range(1, step+1):
        last_state = np.int(fcast_week_state[i-1])
        next_state = np.random.choice(n_states, size=1, p=transition_matrix[last_state,:])[0]
        fcast_week_state[i] = next_state
        #print(next_state)
        
    return fcast_week_state.astype(int)

In [440]:
## map state to the corresponding sales values
def state_to_values(arr_state, arr_mapping):
    arr_fcast = np.zeros(arr_state.shape[0])
    for i in range(arr_state.shape[0]):
        arr_fcast[i] = arr_mapping[arr_state[i]]
    return arr_fcast

In [467]:
## 1000 iterations against the last year prediction
arr_err = []
for i in range(1000):
    arr_err.append(np.sum(state_to_values(sales_forecast(1, M, 51, n_states), 
                                           ly_act_week_sales_fcast) - ly_act_week_sales))
print('Mean Error after 1000 iterations: {}'.format(np.mean(arr_err)))
print('Forecast Error in %: {}'.format(np.mean(arr_err) / np.sum(ly_act_week_sales)))

Mean Error after 1000 iterations: -67.94
Forecast Error in %: -0.013803331978870377


In [461]:
-113.03 / np.sum(ly_act_week_sales)

-0.02296424217797643

In [427]:
np.random.seed(1)
ty_act_week_sales = np.random.randint(low=low+10, high=high-10, size=week_size)

In [428]:
ty_act_week_sales

array([ 47, 150,  82, 147, 143,  89, 154, 139,  81, 144,  35, 188,  30,
       111, 156, 149, 166, 167, 152,  60,  78, 106,  96, 151, 147,  17,
        73,  71,  32,  67,  11, 138,  70,  18, 151, 125, 185, 131,  40,
        81, 141, 159,  59,  67,  13,  34,  53,  86,  36,  62,  90, 119])

In [435]:
ty_act_week_sales_state, ty_act_week_sales_fcast = sales_state_class(ty_act_week_sales, low+10, high-10, n_states)

In [436]:
ty_act_week_sales_fcast

array([ 14.5,  23.5,  32.5,  41.5,  50.5,  59.5,  68.5,  77.5,  86.5,
        95.5, 104.5, 113.5, 122.5, 131.5, 140.5, 149.5, 158.5, 167.5,
       176.5, 185.5])

In [430]:
ty_act_week_sales_state

array([ 4, 14,  8, 14, 14,  8, 15, 13,  8, 14,  3, 18,  2, 11, 15, 14, 16,
       16, 15,  5,  7, 10,  9, 15, 14,  1,  7,  7,  3,  6,  1, 13,  6,  1,
       15, 12, 18, 13,  3,  8, 14, 15,  5,  6,  1,  3,  5,  8,  3,  6,  8,
       11])

In [466]:
## 1000 iterations against this year prediction
arr_err = []
for i in range(1000):
    arr_err.append(np.sum(state_to_values(sales_forecast(1, M, 51, n_states), 
                                           ty_act_week_sales_fcast) - ty_act_week_sales))
print('Mean Error after 1000 iterations: {}'.format(np.mean(arr_err)))
print('Forecast Error in %: {}'.format(np.mean(arr_err) / np.sum(ty_act_week_sales)))

Mean Error after 1000 iterations: -210.812
Forecast Error in %: -0.041360015695507166


##### Notes:
1. The overall forecast error rate is acceptable
2. The calculation of transition matrix could be more sophisticated, e.g. take into account seasonality, week variable and promo

## Forecasting using ARIMA

In [473]:
#see R code

In [472]:
ly_act_week_sales

array([ 37, 140,  72, 137, 133,  79, 192, 144, 129,  71, 134,  25, 178,
        20, 101, 146, 139, 156, 157, 142,  50,  68,  96,  86, 141, 137,
         7,  63,  61,  22,  57,   1, 128,  60,   8, 141, 115, 175, 121,
        30,  71, 131, 198, 149,  49,  57,   3, 196,  24,  43,  76,  26])